In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import lxml 

In [2]:
def scrape_website(departamento, municipio):
    # Inicia el driver (asegúrate de tener ChromeDriver en tu PATH)
    driver = webdriver.Chrome()

    # Navega a la página web
    driver.get("https://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")

    # Espera a que el selector del departamento esté presente
    wait = WebDriverWait(driver, 10)
    element_departamento = wait.until(EC.presence_of_element_located((By.ID, "_ctl0_ContentPlaceHolder1_cmbDepartamento")))

    # Selecciona el departamento
    select_departamento = Select(element_departamento)
    select_departamento.select_by_visible_text(departamento)

    # Espera a que el selector del municipio esté presente
    element_municipio = wait.until(EC.presence_of_element_located((By.ID, "_ctl0_ContentPlaceHolder1_cmbMunicipio")))

    # Selecciona el municipio
    select_municipio = Select(element_municipio)
    select_municipio.select_by_visible_text(municipio)

    # Espera a que el botón esté presente y haz clic en él
    button = wait.until(EC.element_to_be_clickable((By.ID, "_ctl0_ContentPlaceHolder1_IbtnConsultar")))
    button.click()

    # Espera a que la página se recargue después de hacer clic en el botón
    #wait.until(EC.staleness_of(driver.find_element(By.ID, "_ctl0_ContentPlaceHolder1_dgResultado")))

    # Asegúrate de que la tabla se ha cargado completamente
    wait.until(EC.presence_of_element_located((By.ID, "_ctl0_ContentPlaceHolder1_dgResultado")))

    # Ahora que la tabla está cargada, raspa sus datos
    table = driver.find_element(By.ID, "_ctl0_ContentPlaceHolder1_dgResultado")

    # Podrías usar pandas para leer la tabla en un DataFrame
    df = pd.read_html(table.get_attribute('outerHTML'))[0]
    df = df.iloc[1:]
    # Asigna los nombres correctos a las columnas
    df.columns = ['NA','CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 
              'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 
              'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']
    # Eliminar todoas las filas donde el nivel no sea DIVERSIFICADO
    df = df[df['NIVEL'] == 'DIVERSIFICADO']
    
    # Cierra el driver
    driver.quit()

    return df



In [5]:
# Usa la función
departamentos = [ 'HUEHUETENANGO', 'IZABAL','JALAPA','JUTIAPA','PETEN','QUETZALTENANGO','QUICHE','RETALHULEU','SACATEPEQUEZ','SAN MARCOS','SANTA ROSA','SOLOLA','SUCHITEPEQUEZ','TOTONICAPAN','ZACAPA']

for dep in departamentos:
    print(dep)
    df = scrape_website(dep, "TODOS")
    # Guarda el archivo
    
    df.to_csv(f"./data/{dep}.csv", index=False)

HUEHUETENANGO
IZABAL
JALAPA
JUTIAPA
PETEN
QUETZALTENANGO
QUICHE
RETALHULEU
SACATEPEQUEZ
SAN MARCOS
SANTA ROSA
SOLOLA
SUCHITEPEQUEZ
TOTONICAPAN
ZACAPA


-- Tipos de vacios/Nas/NANS
-- Tipos de datos
-- Cantida dde colunmas y filas
-- Solo valores de Diversificados